In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/imdb-dataset-of-top-1000-movies-and-tv-shows/imdb_top_1000.csv


## The Objective of the following notebook is to predict movie on the basis of the name, we will be using imdb dataset of top 1000 moviee.

In [2]:
!pip install rake_nltk
import pandas as pd
import numpy as np
from rake_nltk import Rake
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from numpy import savetxt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 32.3 MB/s eta 0:00:0000:01
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.4
    Uninstalling nltk-3.2.4:
      Successfully uninstalled nltk-3.2.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
preprocessing 0.1.13 requires nltk==3.2.4, but you have nltk 3.8.1 which is incompatible.


In [3]:
df = pd.read_csv("/kaggle/input/imdb-dataset-of-top-1000-movies-and-tv-shows/imdb_top_1000.csv")

In [4]:
df.head()

,Poster_Link,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
0,https://m.media-amazon.com/images/M/MV5BMDFkYT...,The Shawshank Redemption,1994,A,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,"28,341,469"
1,https://m.media-amazon.com/images/M/MV5BM2MyNj...,The Godfather,1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,"134,966,411"
2,https://m.media-amazon.com/images/M/MV5BMTMxNT...,The Dark Knight,2008,UA,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,"534,858,444"
3,https://m.media-amazon.com/images/M/MV5BMWMwMG...,The Godfather: Part II,1974,A,202 min,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,"57,300,000"
4,https://m.media-amazon.com/images/M/MV5BMWU4N2...,12 Angry Men,1957,U,96 min,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845,"4,360,000"


#### Describing the dataset

In [5]:
print('Rows x Columns : ', df.shape[0], 'x', df.shape[1])
print('Features: ', df.columns.tolist())
print('nUnique values:')
print(df.nunique())

Rows x Columns :  1000 x 16
Features:  ['Poster_Link', 'Series_Title', 'Released_Year', 'Certificate', 'Runtime', 'Genre', 'IMDB_Rating', 'Overview', 'Meta_score', 'Director', 'Star1', 'Star2', 'Star3', 'Star4', 'No_of_Votes', 'Gross']
nUnique values:
Poster_Link      1000
Series_Title      999
Released_Year     100
Certificate        16
Runtime           140
Genre             202
IMDB_Rating        17
Overview         1000
Meta_score         63
Director          548
Star1             660
Star2             841
Star3             891
Star4             939
No_of_Votes       999
Gross             823
dtype: int64


### We are using  title plot director , stars and genre for predicting the next movies

In [6]:
df = df.drop(['Poster_Link', 'Released_Year', 'Certificate', 'Runtime','IMDB_Rating', 'Meta_score', 'No_of_Votes', 'Gross'],axis=1)

In [7]:
df.rename(columns = {'Series_Title':'Title'}, inplace = True)

In [8]:
df.rename(columns = {'Overview':'Plot'}, inplace = True)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Title     1000 non-null   object
 1   Genre     1000 non-null   object
 2   Plot      1000 non-null   object
 3   Director  1000 non-null   object
 4   Star1     1000 non-null   object
 5   Star2     1000 non-null   object
 6   Star3     1000 non-null   object
 7   Star4     1000 non-null   object
dtypes: object(8)
memory usage: 62.6+ KB


#### Combining all the actors name under one column 

In [10]:
df['Actors'] = df['Star1']+','+df['Star2']+df['Star3']+df['Star4']

In [11]:
df.head()

,Title,Genre,Plot,Director,Star1,Star2,Star3,Star4,Actors
0,The Shawshank Redemption,Drama,Two imprisoned men bond over a number of years...,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,"Tim Robbins,Morgan FreemanBob GuntonWilliam Sa..."
1,The Godfather,"Crime, Drama",An organized crime dynasty's aging patriarch t...,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,"Marlon Brando,Al PacinoJames CaanDiane Keaton"
2,The Dark Knight,"Action, Crime, Drama",When the menace known as the Joker wreaks havo...,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,"Christian Bale,Heath LedgerAaron EckhartMichae..."
3,The Godfather: Part II,"Crime, Drama",The early life and career of Vito Corleone in ...,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,"Al Pacino,Robert De NiroRobert DuvallDiane Keaton"
4,12 Angry Men,"Crime, Drama",A jury holdout attempts to prevent a miscarria...,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,"Henry Fonda,Lee J. CobbMartin BalsamJohn Fiedler"


In [12]:
df = df.drop(['Star1','Star2','Star3','Star4'],axis=1)

In [13]:
df.head()

,Title,Genre,Plot,Director,Actors
0,The Shawshank Redemption,Drama,Two imprisoned men bond over a number of years...,Frank Darabont,"Tim Robbins,Morgan FreemanBob GuntonWilliam Sa..."
1,The Godfather,"Crime, Drama",An organized crime dynasty's aging patriarch t...,Francis Ford Coppola,"Marlon Brando,Al PacinoJames CaanDiane Keaton"
2,The Dark Knight,"Action, Crime, Drama",When the menace known as the Joker wreaks havo...,Christopher Nolan,"Christian Bale,Heath LedgerAaron EckhartMichae..."
3,The Godfather: Part II,"Crime, Drama",The early life and career of Vito Corleone in ...,Francis Ford Coppola,"Al Pacino,Robert De NiroRobert DuvallDiane Keaton"
4,12 Angry Men,"Crime, Drama",A jury holdout attempts to prevent a miscarria...,Sidney Lumet,"Henry Fonda,Lee J. CobbMartin BalsamJohn Fiedler"


In [14]:
print(df.nunique())

Title        999
Genre        202
Plot        1000
Director     548
Actors       998
dtype: int64


In [15]:
df.isnull().sum()

Title       0
Genre       0
Plot        0
Director    0
Actors      0
dtype: int64

In [16]:
df.describe().T

,count,unique,top,freq
Title,1000,999,Drishyam,2
Genre,1000,202,Drama,85
Plot,1000,1000,Two imprisoned men bond over a number of years...,1
Director,1000,548,Alfred Hitchcock,14
Actors,1000,998,"Mark Hamill,Harrison FordCarrie FisherBilly De...",2


### Using rake(short for Rapid Automatic Keyword Extraction algorithm)  for the text preprocessing.RAKE is a keyword extraction algorithm that extracts those key phrases in a text corpus by determining the frequency of words and their relative occurrence with other words in the corpus.

In [17]:
# to extract key words from Plot to a list
df['Key_words'] = ''   # initializing a new column
r = Rake()   # using Rake to remove stop words

for index, row in df.iterrows():
    r.extract_keywords_from_text(row['Plot'])   # to extract key words 
    key_words_dict_scores = r.get_word_degrees()    # to get dictionary with key words and their similarity scores
    row['Key_words'] = list(key_words_dict_scores.keys())   # to assign it to new column

df

,Title,Genre,Plot,Director,Actors,Key_words
0,The Shawshank Redemption,Drama,Two imprisoned men bond over a number of years...,Frank Darabont,"Tim Robbins,Morgan FreemanBob GuntonWilliam Sa...","[two, imprisoned, men, bond, number, years, fi..."
1,The Godfather,"Crime, Drama",An organized crime dynasty's aging patriarch t...,Francis Ford Coppola,"Marlon Brando,Al PacinoJames CaanDiane Keaton","[organized, crime, dynasty, aging, patriarch, ..."
2,The Dark Knight,"Action, Crime, Drama",When the menace known as the Joker wreaks havo...,Christopher Nolan,"Christian Bale,Heath LedgerAaron EckhartMichae...","[menace, known, joker, wreaks, havoc, chaos, p..."
3,The Godfather: Part II,"Crime, Drama",The early life and career of Vito Corleone in ...,Francis Ford Coppola,"Al Pacino,Robert De NiroRobert DuvallDiane Keaton","[early, life, career, vito, corleone, 1920s, n..."
4,12 Angry Men,"Crime, Drama",A jury holdout attempts to prevent a miscarria...,Sidney Lumet,"Henry Fonda,Lee J. CobbMartin BalsamJohn Fiedler","[jury, holdout, attempts, prevent, miscarriage..."
...,...,...,...,...,...,...
995,Breakfast at Tiffany's,"Comedy, Drama, Romance",A young New York socialite becomes interested ...,Blake Edwards,"Audrey Hepburn,George PeppardPatricia NealBudd...","[young, new, york, socialite, becomes, interes..."
996,Giant,"Drama, Western",Sprawling epic covering the life of a Texas ca...,George Stevens,"Elizabeth Taylor,Rock HudsonJames DeanCarroll ...","[sprawling, epic, covering, life, texas, cattl..."
997,From Here to Eternity,"Drama, Romance, War","In Hawaii in 1941, a private is cruelly punish...",Fred Zinnemann,"Burt Lancaster,Montgomery CliftDeborah KerrDon...","[hawaii, 1941, private, cruelly, punished, box..."
998,Lifeboat,"Drama, War",Several survivors of a torpedoed merchant ship...,Alfred Hitchcock,"Tallulah Bankhead,John HodiakWalter SlezakWill...","[several, survivors, torpedoed, merchant, ship..."


In [18]:
df['Genre'] = df['Genre'].map(lambda x: x.split(','))
df['Actors'] = df['Actors'].map(lambda x: x.split(',')[:3])
df['Director'] = df['Director'].map(lambda x: x.split(','))

# create unique names by merging firstname & surname into one word, & convert to lowercase 
for index, row in df.iterrows():
    row['Genre'] = [x.lower().replace(' ','') for x in row['Genre']]
    row['Actors'] = [x.lower().replace(' ','') for x in row['Actors']]
    row['Director'] = [x.lower().replace(' ','') for x in row['Director']]

#### Generating Word Representations using Bag of Words

In [19]:
df['Bag_of_words'] = ''
columns = ['Genre', 'Director', 'Actors', 'Key_words']

for index, row in df.iterrows():
    words = ''
    for col in columns:
        words += ' '.join(row[col]) + ' '
    row['Bag_of_words'] = words
    
# strip white spaces infront and behind, replace multiple whitespaces (if any)
df['Bag_of_words'] = df['Bag_of_words'].str.strip().str.replace('   ', ' ').str.replace('  ', ' ')

df = df[['Title','Bag_of_words']]

#### Vectorizing BoW and Creating the Similarity Matrix

In [21]:
count = CountVectorizer()
count_matrix = count.fit_transform(df['Bag_of_words'])
count_matrix
cosine_sim = cosine_similarity(count_matrix, count_matrix)

[[1.         0.0571662  0.04850713 ... 0.05423261 0.05170877 0.        ]
 [0.0571662  1.         0.14142136 ... 0.05270463 0.05025189 0.08908708]
 [0.04850713 0.14142136 1.         ... 0.04472136 0.08528029 0.07559289]
 ...
 [0.05423261 0.05270463 0.04472136 ... 1.         0.14301939 0.        ]
 [0.05170877 0.05025189 0.08528029 ... 0.14301939 1.         0.04029115]
 [0.         0.08908708 0.07559289 ... 0.         0.04029115 1.        ]]


In [22]:
indices = pd.Series(df['Title'])

In [25]:
indices = pd.Series(df['Title'])

In [26]:
def recommend(title, cosine_sim = cosine_sim):
    recommended_movies = []
    idx = indices[indices == title].index[0]   # to get the index of the movie title matching the input movie
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)   # similarity scores in descending order
    top_5_indices = list(score_series.iloc[1:6].index)   # to get the indices of top 6 most similar movies
    # [1:6] to exclude 0 (index 0 is the input movie itself)
    
    for i in top_5_indices:   # to append the titles of top 10 similar movies to the recommended_movies list
        recommended_movies.append(list(df['Title'])[i])
        
    return recommended_movies


In [35]:
recommend('Captain America: Civil War')

['Avengers: Infinity War',
 'Star Trek Into Darkness',
 'Captain America: The Winter Soldier',
 'Avengers: Endgame',
 'Jurassic Park']